## 词嵌入

总感觉这个翻译不是特别的好，因为叫Word Embedding。它的目换是希望针对每个单词(word)，找到一个特征的表达，使得这个特征更加显著的包含一些语义信息在里面，比如我们如下面5个单词：
- apple
- bag
- cat
- dog

我们希望在它们被表示为特征时，在特征高维空间中，cat和dog比较近，与apple和bag的距离比较远，因为cat和dog都是动物。也就是我们希望这个特征里包含了“是不是动物”这样latent的语义信息。

## 1-of-N Encoding

我们先看如何用一个vector来表示来个单词，一个经典的方法叫1-of-N Encoding，首先我们先确定一个大的集合，这个集合包含了所有我们关心的单词，假设集合的大小为N，我们给集合中的每个单词一个编号，从零开始。

那么对于一个具体的单词，如果它在集合中的编号为k，则它的向量表示就为$[0,0,\cdots,1,\cdots,0,0]$其中第k位为1，其余位全为0。

## 语义上下文

我们知道单词的意思往往可以通过上下文的语义推断出来，这也是为什么我们在做一些英文阅读理解时，就算某些单词不认识，我们往往也能大概猜到它的意思。

比如我们说“拿起XX”，那么这个“XX”一定是个物件，而且不至于太大，可能是“杯子”、“餐具”、“武器”等等。那么“物件”就是我们对于“XX”分析到的Latent语义。

怎么利用这个上下文的信息呢。我们可以设计一个机器学习模型，通过阅读大量的文章，让它自己的归纳出单词与单词之间的上下文关系，并且找到单词的抽象的特征表达。

### Count based

主导思想是：如果在一个文章中单词$w_i$和$w_j$经常同时出现，那么它们的特征向量$V(w_i)$和$V(w_j)$应该是靠得很近的。我们统计出在所有文章中单词$w_i$和$w_j$同时出现的次数$N_{ij}$，那么我们希望学习到V应该是：$N_{ij}越大，则$$V(w_i)\cdot V(w_j)$越大。

那我们可以把V(w)定义为一个线性模型：
$$V(w) = \sum w\cdot \theta$$

我们的损失函数可以写为：
$$L(\theta) = \sum_{i,j} (V(w_i)\cdot V(w_j) - N_{i,j})^2$$

> [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/): GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

### Prediction Based

`Prediction Based`的方法中呢，我们希望学习一个这样的Neural Network，它的输入是一个句子中若干个(可以是1个或多个)连续的单词，它的输出是这两个单词后面的那个词。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCEfcab24c10055339f6a26d02f0a1a8e55/19120)

当我们把这个Neural Network训练好后，我们取最后某一个fc层的激活值做为单词的feature vector就可以了。

这有一点像让机器学习来做**文字接龙**。

其实上我们可以用上面介绍的Prediction Based方法来做**语言模型(Language Modeling)**，意思就是通过分析大量的文章资料后，然后来求某个句子出现的概率。

$$P("\text{wreck a nice beach}") = P(\text{wreck}|\text{START})P(\text{a}|\text{wreck})P(\text{nice}|\text{a})P(\text{beach}|\text{nice})$$

而$P(\text{wreck}|\text{START}),P(\text{a}|\text{wreck}),P(\text{nice}|\text{a}),P(\text{beach}|\text{nice})$都可以通过我们上面训练出来的Neural Network求出来。

上面图中画的网络结构中我们的网络输入是2个单词，实际我们可以用更多的单词来表示context信息，而且不但我们可以使用左边的连续单词，我们还可以使用右侧的context信息来做预测(这样好像就不能做单词接龙了，只能纯粹做word embedding)。甚至我们可以基于一个单词去预测它两边的context。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCE8e881fa6e5b87b687624b3162cf9716c/19124)

当我们的输入是多个单词的时候，我们可以让多个单词间共享一组权重参数。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCE0d175fb84cd148b1d5b800b9b69034e0/19122)

## 词嵌入的结果分析

经过分析word embedding产生的vector，如果投射到2维平台做可视化的话，会有一些很有意思的现象，如下图：国家和首都之间的距离向量都是比较一致的，每个动词和它的过去式以及过去分词都是3角的关系。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCEae487db9db8245266767a56cae3fc238/19126)

我们可以得到下面这样的关系：
$$V(\text{Rome})-V(\text{Italy})\approx V(\text{Berlin})-V(\text{Germany})$$
$$V(\text{bigger})-V(\text{big})\approx V(\text{hotter})-V(\text{hot})$$
$$V(\text{king})-V(\text{queen})\approx V(\text{uncle})-V(\text{aunt})$$

这个结果，如果应用到图片生成领域，那就将特别有意思了，比如我们通过分析人从小到大的相貌的变化，得到了通过岁数控制相貌的因素，那就可以预测一个现在小朋友，长大后的样子，或现在的成年人变老的样子。

分析出“眼镜”的Latent feature就可给不戴眼镜的人脸图片加眼镜。分析出“口罩”的Latent feature就可给不戴口罩的人脸图片加口罩

## Document Embedding

前面介绍了词嵌入，这一小节简单介绍一下一篇文章如何用一个feature vector来表示呢？　每篇文章的单词数都不相同，我们没办法直接把每个单词的1-of-N Encoding的Feature，然后再加所有单词的feature vector拼在一起。这样每篇文章最后的feature都不一样长了。

一个很好的解决办法叫Bag-of-Word，就是我先选择一个单词集合，然后统计出集合中每个单词在文章中出现的次数，用这样的方式可以把每篇文章都表示为一个固定长度的vector。

这里会有两个问题，一个是如何选择这个单词集合的问题。二是：bag-of-word无法解决两个句子，单词都是一样，但顺序不一样的问题，往往这两个句子的意思相差可能很大。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCE152af7d04e20263043eb5f57c5811649/19128)